In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
spark.version

'3.3.2'

In [3]:
df_fhv = spark.read \
    .option("header", "true") \
    .option("inferSchema","true") \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [5]:
df_fhv.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PUlocationID', IntegerType(), True), StructField('DOlocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [6]:
# schema =  types.StructType([
#     types.StructField('hvfhs_license_num', types.StringType(), True),
#     types.StructField('dispatching_base_num', types.StringType(), True),
#     types.StructField('pickup_datetime', types.TimestampType(), True),
#     types.StructField('dropoff_datetime', types.TimestampType(), True),
#     types.StructField('PULocationID', types.IntegerType(), True),
#     types.StructField('DOLocationID', types.IntegerType(), True),
#     types.StructField('SR_Flag', types.StringType(), True)
# ])
schema =  types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)])

In [7]:
df_fhv = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')



In [10]:
df_fhv = df_fhv.repartition(6)

In [14]:
df_fhv.write.mode('overwrite').parquet("E:\\Test\\data\\raw\\fhv_tripdata\\2019\\10\\")

In [8]:
df = spark.read.parquet('E:\\Test\\data\\raw\\fhv_tripdata\\2019\\10\\')

**Q3**: How many taxi trips were there on February 15?

In [9]:
from pyspark.sql import functions as F

In [10]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [11]:
df.createOrReplaceTempView('fhvhv_2021_02')

In [12]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_02
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



**Q4**: Longest trip for each day

In [22]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [13]:
df \
    .withColumn('duration', (df.dropOff_datetime.cast('long') - df.pickup_datetime.cast('long')) / 3600) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(15) \
    .show()

+-----------+------------------+
|pickup_date|     max(duration)|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333333|
| 2019-10-01| 70128.02805555555|
| 2019-10-17|            8794.0|
| 2019-10-26| 8784.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1056.8266666666666|
| 2019-10-02| 769.2313888888889|
| 2019-10-23| 745.6166666666667|
| 2019-10-03|          745.3825|
| 2019-10-04| 744.6166666666667|
| 2019-10-07| 744.1666666666666|
| 2019-10-05| 697.1808333333333|
| 2019-10-06| 674.0077777777777|
+-----------+------------------+



**Q6**: Least frequent `pickup location zone`

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?


In [17]:
df_zone = spark.read \
    .option("header","true") \
    .option("inferSchema", "true")\
    .csv('taxi_zone_lookup.csv')

In [18]:
df_zone.schema

StructType([StructField('LocationID', IntegerType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [26]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00310|2019-10-14 16:28:32|2019-10-14 16:32:18|         264|         213|   null|                B03047|
|              B00937|2019-10-08 13:14:56|2019-10-08 13:24:40|         264|         243|   null|                B00937|
|              B00837|2019-10-18 17:46:11|2019-10-18 23:29:14|         264|         264|   null|                B00837|
|              B01730|2019-10-19 21:42:52|2019-10-19 22:04:21|         264|         121|   null|                B02096|
|              B00256|2019-10-12 13:51:38|2019-10-12 14:28:14|         264|         264|   null|                B00256|
|              B02437|2019-10-04 08:31:1

In [19]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [25]:
df_zone.registerTempTable('zones')

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [28]:
zones = spark.sql( """
    SELECT * 
    FROM zones
""")

In [32]:
results = df.join(zones, df.PUlocationID == zones.LocationID )
results.registerTempTable('results')

In [33]:
#zone where count()
spark.sql("""
    SELECT zone, COUNT(PUlocationID) AS PICKUP_COUNT
    FROM results
    GROUP BY 1 
    ORDER BY 2 ASC
    LIMIT 5;
""").show()

+--------------------+------------+
|                zone|PICKUP_COUNT|
+--------------------+------------+
|         Jamaica Bay|           1|
|Governor's Island...|           2|
| Green-Wood Cemetery|           5|
|       Broad Channel|           8|
|     Highbridge Park|          14|
+--------------------+------------+

